This notebook benchmarks the core cryptosystem.

In [1]:
#import Pkg
#Pkg.add("Paillier")

include("../src/Paillier.jl")
using Main.Paillier

In [2]:
#using Paillier

In [3]:
PRIMES = Dict(
  1024 => (
    10571726675813419030747017621090503702128756690670143986371236034560692813599426412203713720690798688926328789878099626712676506524453320009379983584398337,
    8719924186273017892025588228882782691633049350158678896348995628952203356872205587084227971878988519308049118884839383574776040781250397380876410886586727
  ),
  2048 => (
    148677972634832330983979593310074301486537017973460461278300587514468301043894574906886127642530475786889672304776052879927627556769456140664043088700743909632312483413393134504352834240399191134336344285483935856491230340093391784574980688823380828143810804684752914935441384845195613674104960646037368551517,
    158741574437007245654463598139927898730476924736461654463975966787719309357536545869203069369466212089132653564188443272208127277664424448947476335413293018778018615899291704693105620242763173357203898195318179150836424196645745308205164116144020613415407736216097185962171301808761138424668335445923774195463
  ),
  3072 => (
    1938773915809394652529298172057636923094290044236022219931405863255082495690032347855183473653298293534659871606091067450014243413240921523500066901919532469137383540029911930316112378749470524457574096696184138193328907669709294330021181329645958261364199493839206628608371081790201428976669130612791242250366444274647280994509241517314575270113602067100993462694264597398469360278093929014852299016988820503145962252559265563066644322641666271029932106499903769,
    1729333955039865312779070302545112355153622867941487879568286271974837188743634878570511341017833291903010427157003137764792255404604305248413507023181875936443712803443246199493193151194550883242324165542306335726099840941650497652937763433867479001683992006377674425496949134225079918435712621197998483974705724043646871299584388204139826795364916574148808511393182730523683601386821553913932390524372176763041155058084219750937709633053529138023382650343754253
  ),
  4096 => (
    28261848413797921050747373790009157475207696600542845759127858334524810394892019587127898078272132395052472769648401097284232757529788630483814946869569002852478396595944244675534735105889321618122474758297393210151468647685747824195258058282068991024204474796049803286388451684224063796077775570765421334497363384354999202901558621538826039758620015278461613759009745213605970355014303833754007641745757347772278772997148831420440876715231559482881530450784440840032110192425362753030300069287596054036239220049206915941278085840877712031976306500253853091618747957831881866466537268377840871519638412119230139490767,
    27320329943961504504949007102163043917650132014364161544882518267176004147597134555329724714487682581532712141674636798503421162701111085999869445909928826112515701752096106236431763411185665259456274656977605110699167613924917533335624140260702229403777488672195828976830865485865262771999567379548038071312755600178975660416672267500965264928789126530448091749895299935378529809339912106262530945639286851709843600337905255291738889952029163800976672706117961832354889533333517431787753974001568371327838181548092337994848828672904504791605188547648282557283655262324819544746399243591989901033723167621115350663317
  ),
)

Dict{Int64,Tuple{BigInt,BigInt}} with 4 entries:
  3072 => (19387739158093946525292981720576369230942900442360222199314058632550…
  1024 => (10571726675813419030747017621090503702128756690670143986371236034560…
  4096 => (28261848413797921050747373790009157475207696600542845759127858334524…
  2048 => (14867797263483233098397959331007430148653701797346046127830058751446…

In [4]:
PLAINTEXTS = Dict(
  "small" => 42,
  "large" => 9601375721773960030826048348718350956180868954786249183055522621772391594913965263068361191091587324151101807311169301869981191762119859865346892157945421998951222949069729370836921713919282283633399891943869137940899827469813950721928452427835958620445001112962904065293585229146038515621140909326729,
)

Dict{String,Signed} with 2 entries:
  "small" => 42
  "large" => 960137572177396003082604834871835095618086895478624918305552262177…

In [5]:
ITERATIONS = 100

PUBKEYS = Dict()
for (keysize, primes) in PRIMES
  PUBKEYS[keysize] = PublicKey(primes[1], primes[2])
end

PRIKEYS = Dict()
for (keysize, primes) in PRIMES
  PRIKEYS[keysize] = PrivateKey(PUBKEYS[keysize], primes[1], primes[2])
end

In [7]:
function bench_encryption(keysize, plaintext)
  pubkey = PUBKEYS[keysize]
  m = PLAINTEXTS[plaintext]

  # Warm up run  
  encrypt(pubkey, m)
    
  total_time = 0.0
  for i = 1:ITERATIONS
    time_elapsed = @elapsed result = encrypt(pubkey, m)
    total_time += time_elapsed
  end

  total_time_in_milliseconds = 1000 * total_time / ITERATIONS
  println("Encryption ($keysize, $plaintext): $(round(total_time_in_milliseconds; digits=2)) ms")
end

for keysize in keys(PRIMES)
  for plaintext in keys(PLAINTEXTS)
    bench_encryption(keysize, plaintext)
  end
end

Encryption (3072, small): 28.21 ms
Encryption (3072, large): 28.11 ms
Encryption (1024, small): 1.86 ms
Encryption (1024, large): 1.87 ms
Encryption (4096, small): 50.1 ms
Encryption (4096, large): 50.07 ms
Encryption (2048, small): 9.28 ms
Encryption (2048, large): 9.38 ms


In [8]:
function bench_decryption(keysize, plaintext)
  pubkey = PUBKEYS[keysize]
  prikey = PRIKEYS[keysize]
  m = PLAINTEXTS[plaintext]
  c = encrypt(pubkey, m)

  decrypt(prikey, c)
  total_time = @elapsed for i = 1:ITERATIONS
    decrypt(prikey, c)
  end

  total_time_in_milliseconds = 1000 * total_time / ITERATIONS
  println("Decryption ($keysize, $plaintext): $(round(total_time_in_milliseconds; digits=2)) ms")
end

for keysize in keys(PRIMES)
  for plaintext in keys(PLAINTEXTS)
    bench_decryption(keysize, plaintext)
  end
end

Decryption (3072, small): 28.0 ms
Decryption (3072, large): 27.91 ms
Decryption (1024, small): 1.27 ms
Decryption (1024, large): 1.25 ms
Decryption (4096, small): 49.03 ms
Decryption (4096, large): 49.05 ms
Decryption (2048, small): 8.93 ms
Decryption (2048, large): 8.83 ms


In [9]:
function bench_addition(keysize, plaintext1, plaintext2)
  pubkey = PUBKEYS[keysize]
  m1 = PLAINTEXTS[plaintext1]
  m2 = PLAINTEXTS[plaintext2]
  c1 = encrypt(pubkey, m1)
  c2 = encrypt(pubkey, m2)

  c1 + c2

  total_time = @elapsed for i = 1:ITERATIONS
    c1 + c2
  end

  total_time_in_milliseconds = 1000 * total_time / ITERATIONS
  println("Addition ($keysize, $plaintext1, $plaintext2): $(round(total_time_in_milliseconds; digits=3)) ms")
end

for keysize in keys(PRIMES)
    for plaintext in keys(PLAINTEXTS)
        for plaintext2 in keys(PLAINTEXTS)
          bench_addition(keysize, plaintext, plaintext2)
        end
    end
end

Addition (3072, small, small): 0.01 ms
Addition (3072, small, large): 0.009 ms
Addition (3072, large, small): 0.009 ms
Addition (3072, large, large): 0.009 ms
Addition (1024, small, small): 0.002 ms
Addition (1024, small, large): 0.002 ms
Addition (1024, large, small): 0.002 ms
Addition (1024, large, large): 0.002 ms
Addition (4096, small, small): 0.015 ms
Addition (4096, small, large): 0.014 ms
Addition (4096, large, small): 0.014 ms
Addition (4096, large, large): 0.014 ms
Addition (2048, small, small): 0.005 ms
Addition (2048, small, large): 0.005 ms
Addition (2048, large, small): 0.005 ms
Addition (2048, large, large): 0.005 ms


In [10]:
function bench_multiplication(keysize, plaintext1, plaintext2)
  pubkey = PUBKEYS[keysize]
  m1 = PLAINTEXTS[plaintext1]
  m2 = PLAINTEXTS[plaintext2]
  c1 = encrypt(pubkey, m1)

  c1 * m2
  total_time = @elapsed for i = 1:ITERATIONS
    c1 * m2
  end

  total_time_in_milliseconds = 1000 * total_time / ITERATIONS
  println("Multiplication ($keysize, $plaintext1, $plaintext2): $(round(total_time_in_milliseconds; digits=3)) ms")
end

for keysize in keys(PRIMES)
    for plaintext in keys(PLAINTEXTS)
        for plaintext2 in keys(PLAINTEXTS)
          bench_multiplication(keysize, plaintext, plaintext2)
        end
    end
end

Multiplication (3072, small, small): 0.078 ms
Multiplication (3072, small, large): 9.379 ms
Multiplication (3072, large, small): 0.081 ms
Multiplication (3072, large, large): 9.455 ms
Multiplication (1024, small, small): 0.012 ms
Multiplication (1024, small, large): 1.228 ms
Multiplication (1024, large, small): 0.015 ms
Multiplication (1024, large, large): 1.239 ms
Multiplication (4096, small, small): 0.112 ms
Multiplication (4096, small, large): 12.391 ms
Multiplication (4096, large, small): 0.111 ms
Multiplication (4096, large, large): 12.491 ms
Multiplication (2048, small, small): 0.038 ms
Multiplication (2048, small, large): 4.342 ms
Multiplication (2048, large, small): 0.038 ms
Multiplication (2048, large, large): 4.353 ms


In [11]:
function bench_keygen(keysize, ITERATIONS=3)
  
  total_time = @elapsed for i = 1:ITERATIONS
    pub, priv = generate_paillier_keypair(keysize)
  end

  total_time_in_milliseconds = 1000 * total_time / ITERATIONS
  println("Keygen ($keysize): $(round(total_time_in_milliseconds; digits=2)) ms")
end

for keysize in keys(PRIMES)
  bench_keygen(keysize)
end

Keygen (3072): 191.6 ms
Keygen (1024): 11.2 ms
Keygen (4096): 861.21 ms
Keygen (2048): 82.06 ms
